In [1]:
from AnnSQL import AnnSQL
import scanpy as sc

In [2]:
adata = sc.datasets.pbmc68k_reduced()
print(adata)

AnnData object with n_obs × n_vars = 700 × 765
    obs: 'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
    var: 'n_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'bulk_labels_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'


In [3]:
asql = AnnSQL(adata=adata)

Time to make var_names unique:  0.042946815490722656
Time to create X table structure:  0.003306865692138672
Time to insert X data:  0.015017032623291016
Finished inserting obs data
Finished inserting var_names data
Finished inserting var data
Finished inserting obsm data
Finished inserting varm data
Finished inserting obsp data


In [7]:
# SELECT example
asql.query("SELECT * FROM X LIMIT 5")

,cell_id,HES4,TNFRSF4,SSU72,PARK7,RBP7,SRM,MAD2L2,AGTRAP,TNFRSF1B,EFHD2,NECAP2,HP1BP3,C1QA,C1QB,HNRNPR,GALE,STMN1,CD52,FGR,ATPIF1,SESN2,EIF3I,LCK,MARCKSL1,SFPQ,PSMB2,MEAF6,NDUFS5,CAP1,SMAP2,C1orf228,PRDX1,TMEM69,SCP2,MAGOH,JAK1,CCBL2,GBP2,CD53,...,FPR1,ZNF600,ZNF524,CTD_3138B18_5,ATP6V1E1,BID,MRPL40,UFD1L,COMT,DGCR6L,SDF2L1,IGLL5,IGLL1,CHCHD10,SMARCB1,MIF,ASCC2,PIK3IP1,HMOX1,EIF3D,IL2RB,LGALS2,EIF3L,ADSL,RBX1,TTC38,TYMP,CCT8,SOD1,PAXBP1,ATP5O,MRPS6,TTC3,U2AF1,CSTB,SUMO3,ITGB2,S100B,PRMT2,MT_ND3
0,AAAGCCTGGCTAAC-1,-0.326,-0.191,-0.728,-0.301,3.386,-0.531,2.016,3.377,4.841,-0.525,-0.525,1.804,0.578,-0.198,1.266,-0.192,-0.307,0.048,-0.606,0.478,-0.07,-0.864,-0.534,-0.504,-0.509,-0.727,-0.646,0.336,-0.017,4.381,-0.251,0.235,-0.252,-0.701,-0.537,-0.684,-0.244,-0.421,0.575,...,-0.27,-0.109,-0.402,-0.122,-0.583,-0.690,-0.487,-0.582,-0.630,-0.507,0.697,-0.133,-0.062,1.777,0.948,0.436,-0.345,-0.418,0.652,0.351,-0.196,-0.413,-0.323,-0.491,-0.719,-0.205,-0.092,-0.694,-0.070,-0.116,-0.146,-0.532,-0.341,0.303,1.404,4.294,0.519,-0.21,-0.636,4.011
1,AAATTCGATGCACA-1,1.171,-0.191,0.795,-1.200,-0.174,-0.531,1.889,-0.486,-0.459,-0.525,1.287,-0.420,0.954,2.367,1.170,-0.192,-0.307,0.266,0.629,-0.802,-0.07,-0.864,-0.534,0.873,-0.509,0.645,-0.646,0.274,0.856,1.845,-0.251,-0.796,-0.252,-0.701,1.581,-0.684,-0.244,-0.421,-0.282,...,-0.27,-0.109,-0.402,-0.122,-0.583,2.992,1.327,-0.582,0.991,-0.507,0.636,-0.133,-0.062,-0.687,-0.497,0.373,-0.345,-0.418,-0.366,1.496,-0.196,-0.413,0.478,-0.491,-0.719,-0.205,1.089,-0.694,-0.120,-0.116,-1.136,-0.532,-0.341,-0.905,2.849,-0.585,1.172,-0.21,2.630,-0.490
2,AACACGTGGTCTTT-1,-0.326,-0.191,0.483,-1.200,-0.174,-0.531,-0.451,0.971,-0.459,2.286,-0.525,1.258,-0.310,-0.198,-0.602,-0.192,-0.307,-0.462,1.358,-0.802,-0.07,0.205,-0.534,-0.504,-0.509,0.364,-0.646,-0.881,-0.256,-0.445,-0.251,-0.796,-0.252,0.480,2.831,1.528,-0.244,0.816,-0.440,...,-0.27,-0.109,-0.402,-0.122,-0.583,-0.690,-0.487,-0.582,-0.630,-0.507,-0.502,-0.133,-0.062,-0.687,-0.497,-0.782,-0.345,-0.418,-0.366,0.037,5.397,-0.413,0.131,1.310,-0.719,-0.205,-0.733,1.747,0.428,-0.116,-1.136,2.606,-0.341,-0.905,-0.455,-0.585,0.722,-0.21,0.663,-0.490
3,AAGTGCACGTGCTA-1,-0.326,-0.191,1.134,-0.157,-0.174,-0.531,-0.451,-0.486,-0.459,-0.525,-0.525,9.900,-0.310,-0.198,1.565,-0.192,2.270,1.258,-0.606,-0.802,-0.07,-0.864,0.741,-0.504,2.193,2.630,1.627,-0.881,-0.990,-0.445,-0.251,-0.796,-0.252,1.114,-0.537,-0.684,-0.244,1.481,2.728,...,-0.27,-0.109,-0.402,-0.122,-0.583,-0.690,-0.487,-0.582,-0.630,-0.507,-0.502,-0.133,-0.062,-0.687,-0.497,-0.782,-0.345,-0.418,-0.366,2.036,-0.196,-0.413,-0.180,-0.491,-0.719,-0.205,-0.733,-0.694,1.221,-0.116,1.161,-0.532,-0.341,-0.905,-0.119,-0.585,0.766,-0.21,-0.636,-0.490
4,ACACGAACGGAGTG-1,-0.326,-0.191,-0.728,-0.607,-0.174,-0.531,-0.451,0.787,-0.459,1.932,-0.525,1.046,-0.310,-0.198,0.630,-0.192,-0.307,-0.546,1.110,1.730,-0.07,1.005,-0.534,-0.504,-0.509,-0.727,-0.646,-0.881,0.293,-0.445,-0.251,-0.796,-0.252,1.363,2.407,0.283,-0.244,-0.421,-0.518,...,-0.27,-0.109,-0.402,-0.122,-0.583,-0.690,-0.487,-0.582,-0.630,-0.507,-0.502,-0.133,-0.062,-0.687,-0.497,0.824,-0.345,-0.418,0.306,-0.926,-0.196,3.847,1.138,-0.491,0.213,-0.205,-0.310,-0.694,-1.048,-0.116,-1.136,0.839,1.679,-0.108,-0.534,-0.585,-0.007,-0.21,-0.636,-0.490


In [8]:
# Filter example
asql.query("SELECT * FROM X WHERE HES4 > 4")

,cell_id,HES4,TNFRSF4,SSU72,PARK7,RBP7,SRM,MAD2L2,AGTRAP,TNFRSF1B,EFHD2,NECAP2,HP1BP3,C1QA,C1QB,HNRNPR,GALE,STMN1,CD52,FGR,ATPIF1,SESN2,EIF3I,LCK,MARCKSL1,SFPQ,PSMB2,MEAF6,NDUFS5,CAP1,SMAP2,C1orf228,PRDX1,TMEM69,SCP2,MAGOH,JAK1,CCBL2,GBP2,CD53,...,FPR1,ZNF600,ZNF524,CTD_3138B18_5,ATP6V1E1,BID,MRPL40,UFD1L,COMT,DGCR6L,SDF2L1,IGLL5,IGLL1,CHCHD10,SMARCB1,MIF,ASCC2,PIK3IP1,HMOX1,EIF3D,IL2RB,LGALS2,EIF3L,ADSL,RBX1,TTC38,TYMP,CCT8,SOD1,PAXBP1,ATP5O,MRPS6,TTC3,U2AF1,CSTB,SUMO3,ITGB2,S100B,PRMT2,MT_ND3
0,GATCTACTTTATCC-3,4.430,-0.191,0.885,-0.297,-0.174,0.995,-0.451,-0.486,6.641,3.219,1.393,-0.420,0.136,-0.198,-0.602,-0.192,-0.307,-1.127,-0.606,-0.802,-0.07,-0.864,-0.534,-0.504,-0.509,-0.727,-0.646,-0.881,1.943,1.980,-0.251,-0.796,-0.252,0.871,-0.537,-0.684,-0.244,1.226,-1.055,...,-0.270,-0.109,-0.402,-0.122,-0.583,2.234,1.434,-0.582,-0.630,-0.507,-0.502,-0.133,-0.062,4.264,-0.497,-0.782,3.148,-0.418,0.657,-0.926,-0.196,-0.413,-0.319,-0.491,3.540,-0.205,3.126,0.931,-0.065,-0.116,-1.136,-0.532,-0.341,-0.905,1.416,1.049,2.078,-0.21,-0.636,-0.490
1,GTAGCTGATTCCAT-4,6.504,-0.191,-0.728,0.356,-0.174,-0.531,-0.451,-0.486,-0.459,1.087,-0.525,-0.420,2.380,-0.198,-0.602,-0.192,-0.307,-0.619,0.520,0.306,-0.07,-0.864,-0.534,0.752,-0.509,-0.727,-0.646,0.172,0.694,1.644,-0.251,-0.796,-0.252,0.653,-0.537,-0.684,-0.244,-0.421,1.062,...,-0.270,-0.109,-0.402,-0.122,-0.583,0.150,-0.487,-0.582,0.849,1.501,0.536,-0.133,-0.062,1.446,-0.497,-0.782,-0.345,-0.418,1.397,-0.926,-0.196,-0.413,-0.443,-0.491,-0.719,-0.205,0.376,0.706,1.491,-0.116,-0.279,-0.532,-0.341,0.141,1.071,-0.585,-0.357,-0.21,0.854,-0.490
2,ATGTTCACGTACGT-6,5.209,-0.191,0.398,0.061,4.820,-0.531,1.279,-0.486,-0.459,-0.525,0.814,-0.420,3.427,2.173,2.019,-0.192,0.082,0.109,2.133,-0.802,-0.07,-0.864,-0.534,2.550,-0.509,-0.727,0.729,-0.027,-0.990,1.248,-0.251,-0.796,-0.252,-0.701,-0.537,0.344,-0.244,0.729,0.088,...,1.981,-0.109,-0.402,-0.122,0.796,2.713,-0.487,-0.582,-0.630,-0.507,0.339,-0.133,-0.062,0.177,-0.497,-0.782,-0.345,-0.418,-0.366,-0.926,-0.196,-0.413,-0.590,-0.491,-0.719,-0.205,1.962,-0.694,-0.362,-0.116,0.253,0.926,-0.341,1.638,1.244,0.556,-0.484,-0.21,0.571,-0.490
3,GCATTGGAATGCCA-6,4.047,-0.191,2.238,-1.200,-0.174,-0.531,1.828,-0.486,-0.459,-0.525,1.239,-0.420,-0.310,-0.198,-0.602,-0.192,-0.307,0.229,3.001,0.380,-0.07,0.445,-0.534,-0.504,-0.509,0.609,-0.646,0.243,-0.091,1.784,-0.251,0.157,-0.252,-0.701,-0.537,0.670,-0.244,-0.421,-1.055,...,-0.270,-0.109,-0.402,-0.122,-0.583,0.206,-0.487,-0.582,-0.630,1.635,-0.502,-0.133,-0.062,0.451,-0.497,0.343,-0.345,-0.418,-0.366,1.432,-0.196,-0.413,2.082,-0.491,1.891,-0.205,0.450,-0.694,-1.048,-0.116,-0.221,-0.532,-0.341,0.211,2.745,0.917,1.828,-0.21,-0.636,-0.490
4,ACGGAACTAGTCGT-7,4.703,-0.191,-0.728,-0.484,-0.174,-0.531,1.514,1.053,2.357,-0.525,-0.525,-0.420,0.044,-0.198,-0.602,-0.192,-0.307,-0.659,3.542,-0.802,-0.07,0.265,-0.534,-0.504,-0.509,-0.727,-0.646,0.089,-0.215,1.478,-0.251,-0.796,-0.252,0.546,-0.537,-0.684,-0.244,-0.421,-0.406,...,-0.270,-0.109,1.682,-0.122,-0.583,1.629,-0.487,-0.582,0.731,-0.507,0.453,-0.133,-0.062,1.276,-0.497,-0.782,-0.345,-0.418,2.068,0.091,-0.196,-0.413,-1.216,-0.491,0.407,-0.205,0.798,-0.694,1.290,-0.116,-0.347,-0.532,-0.341,-0.905,0.900,-0.585,2.051,-0.21,-0.636,-0.490
5,ATGTTGCTGGGCAA-7,5.649,-0.191,0.792,-0.349,-0.174,-0.531,-0.451,-0.486,2.886,-0.525,-0.525,1.685,0.530,-0.198,-0.602,-0.192,-0.307,-0.571,1.858,1.621,-0.07,-0.864,-0.534,0.870,-0.509,-0.727,-0.646,2.575,-0.990,1.840,2.836,0.180,-0.252,-0.701,-0.537,-0.684,-0.244,-0.421,2.032,...,-0.270,-0.109,-0.402,-0.122,-0.583,1.147,-0.487,-0.582,-0.630,-0.507,0.633,-0.133,-0.062,0.479,-0.497,-0.782,-0.345,-0.418,2.525,2.699,-0.196,-0.413,0.474,-0.491,5.969,-0.205,1.085,-0.694,-0.122,-0.116,-1.136,1.436,-0.341,2.526,-0.296,0.954,1.167,-0.21,2.623,1.641
6,CAGAAGCTTAGTCG-7,6.444,-0.191,-0.728,-1.200,-0.174,-0.531,-0.451,1.171,1.057,-0.525,-0.525,-0.420,-0.310,-0.198,-0.602,-0.192,-0.307,-0.875,0.511,-0.802,-0.07,-0.864,-0.534,-0.504,-0.509,-0.727,1.035,0.16

In [9]:
# Filter by cell type
asql.query("SELECT * FROM adata WHERE bulk_labels = 'Dendritic' AND percent_mito > 0.025 ")

,cell_id,bulk_labels,n_genes,percent_mito,n_counts,S_score,G2M_score,phase,louvain,cell_id_1,HES4,TNFRSF4,SSU72,PARK7,RBP7,SRM,MAD2L2,AGTRAP,TNFRSF1B,EFHD2,NECAP2,HP1BP3,C1QA,C1QB,HNRNPR,GALE,STMN1,CD52,FGR,ATPIF1,SESN2,EIF3I,LCK,MARCKSL1,SFPQ,PSMB2,MEAF6,NDUFS5,CAP1,SMAP2,...,FPR1,ZNF600,ZNF524,CTD_3138B18_5,ATP6V1E1,BID,MRPL40,UFD1L,COMT,DGCR6L,SDF2L1,IGLL5,IGLL1,CHCHD10,SMARCB1,MIF,ASCC2,PIK3IP1,HMOX1,EIF3D,IL2RB,LGALS2,EIF3L,ADSL,RBX1,TTC38,TYMP,CCT8,SOD1,PAXBP1,ATP5O,MRPS6,TTC3,U2AF1,CSTB,SUMO3,ITGB2,S100B,PRMT2,MT_ND3
0,AAATTCGATGCACA-1,Dendritic,1080,0.027458,2695.0,0.067026,-0.889498,S,1,AAATTCGATGCACA-1,1.171,-0.191,0.795,-1.200,-0.174,-0.531,1.889,-0.486,-0.459,-0.525,1.287,-0.420,0.954,2.367,1.170,-0.192,-0.307,0.266,0.629,-0.802,-0.07,-0.864,-0.534,0.873,-0.509,0.645,-0.646,0.274,0.856,1.845,...,-0.270,-0.109,-0.402,-0.122,-0.583,2.992,1.327,-0.582,0.991,-0.507,0.636,-0.133,-0.062,-0.687,-0.497,0.373,-0.345,-0.418,-0.366,1.496,-0.196,-0.413,0.478,-0.491,-0.719,-0.205,1.089,-0.694,-0.120,-0.116,-1.136,-0.532,-0.341,-0.905,2.849,-0.585,1.172,-0.21,2.630,-0.490
1,ATGGACACTCGTTT-1,Dendritic,1198,0.028549,3783.0,-0.155325,-1.028952,G1,2,ATGGACACTCGTTT-1,-0.326,-0.191,-0.728,-1.200,-0.174,1.522,-0.451,-0.486,-0.459,-0.525,-0.525,-0.420,-0.310,-0.198,0.661,-0.192,-0.307,-0.730,-0.606,-0.802,-0.07,0.094,-0.534,-0.504,-0.509,-0.727,-0.646,-0.881,-0.990,1.186,...,-0.270,-0.109,-0.402,-0.122,2.075,0.622,-0.487,-0.582,0.525,-0.507,-0.502,-0.133,-0.062,0.145,-0.497,0.041,-0.345,-0.418,-0.366,-0.063,-0.196,5.045,1.197,-0.491,0.236,-0.205,-0.733,-0.694,-1.048,-0.116,-1.136,-0.532,1.730,1.545,0.040,-0.585,1.062,-0.21,-0.636,-0.490
2,CATCCCGATCTCAT-1,Dendritic,1105,0.025213,3054.0,-0.091713,-0.867561,G1,5,CATCCCGATCTCAT-1,2.316,-0.191,0.616,-0.448,-0.174,-0.531,-0.451,1.131,-0.459,-0.525,-0.525,-0.420,-0.310,-0.198,0.962,-0.192,-0.307,0.102,5.932,1.341,-0.07,-0.864,-0.534,-0.504,-0.509,0.484,-0.646,1.157,-0.990,-0.445,...,-0.270,-0.109,-0.402,-0.122,-0.583,0.122,-0.487,0.988,0.800,1.435,0.502,-0.133,-0.062,0.344,0.713,-0.782,-0.345,-0.418,0.487,-0.926,2.907,-0.413,-0.468,-0.491,0.464,-0.205,0.339,-0.694,-0.229,-0.116,-0.307,-0.532,-0.341,-0.905,-0.387,-0.585,2.207,-0.21,-0.636,-0.490
3,AACCACGACGTTGA-3,Dendritic,1190,0.029216,3012.0,-0.232771,-0.777265,G1,5,AACCACGACGTTGA-3,-0.326,-0.191,0.635,0.326,-0.174,-0.531,1.643,1.154,1.041,-0.525,-0.525,1.468,-0.310,-0.198,-0.602,-0.192,-0.307,-0.130,1.604,-0.802,-0.07,-0.864,0.399,-0.504,-0.509,-0.727,-0.646,-0.881,0.662,-0.445,...,-0.270,-0.109,-0.402,-0.122,-0.583,-0.690,-0.487,-0.582,0.820,-0.507,-0.502,-0.133,-0.062,0.359,-0.497,0.252,-0.345,-0.418,0.499,-0.926,-0.196,-0.413,-1.216,-0.491,2.880,-0.205,-0.733,-0.694,-1.048,-0.116,-1.136,1.233,-0.341,-0.905,1.029,-0.585,0.285,-0.21,0.825,-0.490
4,AGGCAACTTGGTAC-3,Dendritic,1233,0.029189,3563.0,-0.151107,-0.800019,G1,5,AGGCAACTTGGTAC-3,1.939,-0.191,0.424,-1.200,-0.174,-0.531,1.319,0.900,-0.459,0.812,-0.525,-0.420,-0.310,-0.198,-0.602,-0.192,-0.307,0.770,3.130,0.116,-0.07,-0.864,1.831,-0.504,-0.509,0.311,0.760,-0.881,-0.990,-0.445,...,-0.270,-0.109,-0.402,-0.122,0.828,-0.690,-0.487,0.764,0.596,-0.507,-0.502,-0.133,-0.062,-0.687,-0.497,-0.782,-0.345,-0.418,-0.366,-0.926,-0.196,-0.413,-1.216,-0.491,-0.719,3.235,0.186,-0.694,-1.048,-0.116,0.995,0.960,1.858,-0.038,0.109,-0.585,-0.472,-0.21,-0.636,-0.490
5,AACTCACTGGGTGA-5,Dendritic,1194,0.027857,3841.0,0.159842,-0.919129,S,2,AACTCACTGGGTGA-5,-0.326,-0.191,-0.728,0.595,-0.174,0.480,-0.451,-0.486,-0.459,-0.525,0.746,1.060,-0.310,-0.198,0.642,-0.192,-0.307,-1.127,-0.606,0.902,-0.07,0.079,-0.534,-0.504,-0.509,-0.727,0.658,-0.881,-0.342,1.162,...,-0.270,-0.109,-0.402,-0.122,0.726,-0.044,-0.487,-0.582,2.782,-0.507,0.296,-0.133,-0.062,-0.687,-0.497,-0.782,-0.345,-0.418,-0.366,-0.076,-0.196,1.738,-1.216,-0.491,1.162,-0.205,0.120,-0.694,-1.048,-0.116,0.841,-0.532,1.699,-0.905,0.023,-0.585,-0.512,-0.21,-0.636,-0.490
6,CGCTACTGGACAGG-6,Dendritic,1241,0.029662,3405.0,-0.187843,-0.945558,G1,2,CGCTACTGGACAG

In [16]:
# Count total cells by cell type
asql.query("""
    SELECT bulk_labels, COUNT(*) as total
    FROM obs
    GROUP BY bulk_labels
    ORDER BY total DESC
""")

,bulk_labels,total
0,Dendritic,240
1,CD14+ Monocyte,129
2,CD19+ B,95
3,CD4+/CD25 T Reg,68
4,CD8+ Cytotoxic T,54
5,CD8+/CD45RA+ Naive Cytotoxic,43
6,CD56+ NK,31
7,CD4+/CD45RO+ Memory,19
8,CD34+,13
9,CD4+/CD45RA+/CD25- Naive T,8


In [17]:
# Show all tables in database
asql.query("SHOW tables;")

,name
0,X
1,X
2,adata
3,obs
4,obsm_X_pca
5,obsm_X_umap
6,obsp_connectivities
7,obsp_distances
8,uns_raw
9,var
